In [ ]:
from pyspark.sql.functions import *

# Load the raw data from the data lake 

In [ ]:
def load_data():
    df_orders = spark.read.csv('abfss://<container>@<account-name>.dfs.core.windows.net/<path>/orders.csv', header='true', inferSchema='true')
    return df_orders

# Clean the data

In [ ]:
def remove_duplicate_orders(df):
    return df.dropDuplicates(["OrderId"])

# Calculate reporting metrics

In [ ]:
def calculate_sales_by_region(df):
    return df_orders \
        .select("Region", "TotalPrice") \
        .groupBy("Region") \
        .sum("TotalPrice") \
        .withColumnRenamed("sum(TotalPrice)", "TotalSales")

# Save the output

In [ ]:
def save_output(df):
    df.repartition(1) \
        .write.mode("overwrite") \
        .option("header", "true") \
        .csv('abfss://<container>@<account-name>.dfs.core.windows.net/<path>/output/')

    df.show()

In [ ]:
df_orders = load_data()
df_unique_orders = remove_duplicate_orders(df_orders)
df_sales_by_region = calculate_sales_by_region(df_unique_orders)
save_output(df_sales_by_region)